In [13]:
# 구글 드라이브 마운트(cjyjob1993@gmail.com)
from google.colab import drive
drive.mount('/content/drive')

# lib 디렉토리를을 환경 변수에 추가
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/myCode/lib')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# 라이브러리 임포트
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, losses
from tensorflow.keras.metrics import Precision, Recall
from keras.callbacks import Callback, ModelCheckpoint
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
# 커스텀 라이브러리 임포트
import tensorflow_fixSeed
from pandas_readDataFrame import df2, df2_label
from func_debug_log import debug
from func_pandas_exploreDataFrame import exploreDataFrame
from keras_getF1scoreInNeuralNetwork import get_f1

In [15]:
# debug_log (0 : 미출력, 1 : 출력)
debug_flag = 1

In [16]:
if debug_flag == 1 : debug('DataFrame 탐색', __name__)
exploreDataFrame(df2)
exploreDataFrame(df2_label)

2022.12.02 02:13:21 __main__ DataFrame 탐색
(1077, 69) 

        0        1        2       3       4        5        6       7   \
0  0.50364  0.74719  0.43018  2.8163  5.9902  0.21883  0.17757  2.3737   
1  0.51462  0.68906  0.41177  2.7895  5.6850  0.21164  0.17282  2.2921   
2  0.50918  0.73025  0.41831  2.6872  5.6221  0.20901  0.17572  2.2833   

        8       9   ...       59       60       61      62      63       64  \
0  0.23222  1.7509  ...  0.14276  0.43096  0.24754  1.6033  2.0149  0.10823   
1  0.22697  1.5964  ...  0.14204  0.45716  0.25763  1.6717  2.0046  0.10975   
2  0.23025  1.5613  ...  0.14244  0.51047  0.25534  1.6635  2.0168  0.10820   

        65       66       67      68  
0  1.04500  0.83156  0.18885  1.6757  
1  1.00990  0.84927  0.20040  1.7436  
2  0.99685  0.84671  0.19368  1.9264  

[3 rows x 69 columns] 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1077 entries, 0 to 1076
Data columns (total 69 columns):
 #   Column  Non-Null Count  Dtype  
---  -

In [17]:
if debug_flag == 1 : debug('train, test 데이터 분할', __name__)
df2_label = df2_label - 1
df2_label.astype(object)
X_train, X_test, y_train, y_test = train_test_split(df2, df2_label, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

2022.12.02 02:13:21 __main__ train, test 데이터 분할


In [21]:
if debug_flag == 1 : debug('신경망 모델 빌드 함수 선언', __name__)

input_shape = X_train.shape[1] # X_train의 input_shape 변수 바인딩
def mlp_v2():
    mdl = Sequential()
    mdl.add(Dense(12,  input_dim = 69, activation='relu',input_shape=(input_shape,)))
    mdl.add(Dropout(0.5))
    mdl.add(Dense(128, activation='relu'))
    mdl.add(Dropout(0.5))
    mdl.add(Dense(1, activation='sigmoid'))
    mdl.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=[get_f1])
    mdl.summary()
    return mdl

2022.12.02 02:13:21 __main__ 신경망 모델 빌드 함수 선언


In [22]:
if debug_flag == 1 : debug('신경망 모델 학습 checkpoint 설정', __name__)

mode_path = '../models/mlp_v2.h5'
callbacks = [ModelCheckpoint(filepath=mode_path, save_best_only=True)]

2022.12.02 02:13:21 __main__ 신경망 모델 학습 checkpoint 설정


In [23]:
if debug_flag == 1 : debug('신경망 모델 학습', __name__)
estimator = KerasClassifier(build_fn = mlp_v2, epochs=5, batch_size=128)
history = estimator.fit(X_train, y_train, validation_data = (X_val, y_val), callbacks=callbacks)

2022.12.02 02:13:21 __main__ 신경망 모델 학습
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                840       
                                                                 
 dropout (Dropout)           (None, 12)                0         
                                                                 
 dense_1 (Dense)             (None, 128)               1664      
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 2,633
Trainable params: 2,633
Non-trainable params: 0
_________________________________________________________________


<ipython-input-23-6a45089499ce>:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn = mlp_v2, epochs=5, batch_size=128)


Epoch 1/5
6/6 [==============================] - 1s 57ms/step - loss: 0.9023 - get_f1: 0.5649 - val_loss: 0.7719 - val_get_f1: 0.5439
Epoch 2/5
6/6 [==============================] - 0s 10ms/step - loss: 0.8751 - get_f1: 0.5840 - val_loss: 0.7717 - val_get_f1: 0.5439
Epoch 3/5
6/6 [==============================] - 0s 10ms/step - loss: 0.8516 - get_f1: 0.5955 - val_loss: 0.7715 - val_get_f1: 0.5439
Epoch 4/5
6/6 [==============================] - 0s 11ms/step - loss: 0.8374 - get_f1: 0.6022 - val_loss: 0.7713 - val_get_f1: 0.5439
Epoch 5/5
6/6 [==============================] - 0s 10ms/step - loss: 0.8519 - get_f1: 0.5966 - val_loss: 0.7711 - val_get_f1: 0.5439
